We make sure the export statements are generated through the dbt project. Just scan edge/node files.

In [1]:
import pathlib

WHERE_IS_DBT_PROJECT_SQLS = "../../omdb_manual_dataset/omdb_dbt_project/models/gold_business_graph/*.sql"

table_names = [f.name.split('.')[0] for f in pathlib.Path().glob(WHERE_IS_DBT_PROJECT_SQLS)]

WHERE_TO_EXPORT_CSV_FILES = 'C:/work/omdb_graphdb_article/kuzudb_version/export_elt'

p = pathlib.Path(WHERE_TO_EXPORT_CSV_FILES)
p.mkdir(exist_ok=True)

for json_file in p.glob('*.json'):
    print(json_file)
    json_file.unlink()


C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_category_has_parent.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_category_has_root.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_content_has_cast.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_content_has_category.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_content_has_keyword.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_episode_has_season.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_episode_has_series.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_job_has_cast.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_business_graph_edge_movie_has_movieseries.json
C:\work\omdb_graphdb_article\kuzudb_version\export_elt\gold_busine

In [2]:
table_names

['gold_business_graph_edge_category_has_parent',
 'gold_business_graph_edge_category_has_root',
 'gold_business_graph_edge_content_has_cast',
 'gold_business_graph_edge_content_has_category',
 'gold_business_graph_edge_content_has_keyword',
 'gold_business_graph_edge_episode_has_season',
 'gold_business_graph_edge_episode_has_series',
 'gold_business_graph_edge_job_has_cast',
 'gold_business_graph_edge_movie_has_movieseries',
 'gold_business_graph_edge_person_has_cast',
 'gold_business_graph_edge_season_has_series',
 'gold_business_graph_edge_series_has_movieseries',
 'gold_business_graph_node_cast',
 'gold_business_graph_node_category',
 'gold_business_graph_node_character',
 'gold_business_graph_node_content',
 'gold_business_graph_node_image',
 'gold_business_graph_node_job',
 'gold_business_graph_node_person']

We export to json node/edge tables

In [3]:
!python -m pip install psycopg2-binary

In [4]:
import psycopg2
import io
# Connect to an existing database

for table_name in table_names:

  print("processing "+table_name)

  sql_query = f"""
  COPY (
    SELECT row_to_json(temp_data) FROM (
      SELECT
        *
      FROM dbt.{table_name}
    ) temp_data
  ) TO STDOUT;
  """
  with psycopg2.connect("dbname=postgres user=postgres password=postgres") as conn:
      with conn.cursor() as cur:
        with io.StringIO() as f:
          cur.copy_expert(sql_query, f)
          contents = f.getvalue()
        result = "["+(",".join(contents.split("\n")))
        result = result[:-1]+"]"

  with open(f"{WHERE_TO_EXPORT_CSV_FILES}/{table_name}.json", "w", encoding="utf8") as ffile:
    ffile.write(result)




processing gold_business_graph_edge_category_has_parent
processing gold_business_graph_edge_category_has_root
processing gold_business_graph_edge_content_has_cast
processing gold_business_graph_edge_content_has_category
processing gold_business_graph_edge_content_has_keyword
processing gold_business_graph_edge_episode_has_season
processing gold_business_graph_edge_episode_has_series
processing gold_business_graph_edge_job_has_cast
processing gold_business_graph_edge_movie_has_movieseries
processing gold_business_graph_edge_person_has_cast
processing gold_business_graph_edge_season_has_series
processing gold_business_graph_edge_series_has_movieseries
processing gold_business_graph_node_cast
processing gold_business_graph_node_category
processing gold_business_graph_node_character
processing gold_business_graph_node_content
processing gold_business_graph_node_image
processing gold_business_graph_node_job
processing gold_business_graph_node_person
